In [2]:
import warnings
warnings.showwarning = lambda *a, **k: None
warnings.filterwarnings("ignore")

print(" Warnings disabled")

 Warnings disabled


In [5]:
!pip install -q datasets sentence-transformers faiss-cpu requests beautifulsoup4 duckduckgo-search

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Ammu\\Downloads\\Python\\venv\\Lib\\site-packages\\torch\\_export\\error.py'
Check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Ammu\\Downloads\\Python\\venv\\Lib\\site-packages\\sklearn\\_config.py'
Check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import faiss
import requests
import re

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS

ModuleNotFoundError: No module named 'datasets'


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
HEADERS = {
    "User-Agent": "AutoRAG-Demo/1.0 (contact: demo@example.com)"
}

In [5]:
dataset = load_dataset("wikitext", "wikitext-103-v1", split="train")

texts = []
for item in dataset:
    t = item["text"].strip()
    if t:
        texts.append(t)
    if len(texts) >= 1000:
        break

print("Loaded texts:", len(texts))

README.md: 0.00B [00:00, ?B/s]

wikitext-103-v1/test-00000-of-00001.parq(…):   0%|          | 0.00/722k [00:00<?, ?B/s]

wikitext-103-v1/train-00000-of-00002.par(…):   0%|          | 0.00/156M [00:00<?, ?B/s]

wikitext-103-v1/train-00001-of-00002.par(…):   0%|          | 0.00/156M [00:00<?, ?B/s]

wikitext-103-v1/validation-00000-of-0000(…):   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Loaded texts: 1000


In [6]:
def chunk_text(text, size=500, overlap=50):
    chunks = []
    i = 0
    while i < len(text):
        chunks.append(text[i:i+size])
        i += size - overlap
    return chunks


In [7]:
base_chunks = []
for t in texts:
    base_chunks.extend(chunk_text(t))

print("Base chunks:", len(base_chunks))


Base chunks: 1697


In [8]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

base_embeddings = embedder.encode(
    base_chunks,
    batch_size=32,
    show_progress_bar=True
)

base_embeddings = np.array(base_embeddings).astype("float32")
faiss.normalize_L2(base_embeddings)

base_index = faiss.IndexFlatIP(base_embeddings.shape[1])
base_index.add(base_embeddings)

print("Base FAISS vectors:", base_index.ntotal)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Base FAISS vectors: 1697


In [ ]:
BAD_PATTERNS = [
    r"accept all cookies",
    r"manage preferences",
    r"privacy policy",
    r"cookie",
    r"we value your privacy",
    r"essential cookies",
    r"skip to main",
    r"skip to content",
    r"menu",
    r"navigation",
    r"subscribe",
    r"newsletter",
    r"sign up",
    r"log in",
    r"register",
    r"follow us",
    r"share on",
    r"tweet",
    r"facebook",
    r"instagram",
    r"linkedin",
    r"©.*?copyright",
    r"all rights reserved",
]

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # Remove excessive whitespace and newlines first
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\r+', ' ', text)
    
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    
    # Remove common web noise patterns (case-insensitive)
    for pattern in BAD_PATTERNS:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    
    # Remove excessive special characters but keep basic punctuation
    text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)\'\"\&]', ' ', text)
    
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Keep only substantial text (more than 50 chars for better quality)
    if len(text) < 50:
        return ""
    
    return text


In [ ]:
def clean_answer(text, max_sentences=5):
    if not text:
        return ""
    
    # Split into sentences (better than splitting on periods)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 30]
    
    # Take first few meaningful sentences
    result = ". ".join(sentences[:max_sentences])
    
    # Clean up any remaining artifacts
    result = re.sub(r'\s+', ' ', result)
    result = result.strip()
    
    # Ensure it ends with punctuation
    if result and not result[-1] in '.!?':
        result += "."
    
    return result


In [ ]:
def retrieve_from(index, chunks, query, k=3):
    if index is None or len(chunks) == 0:
        return [], 0.0
    
    q = embedder.encode([query]).astype("float32")
    faiss.normalize_L2(q)

    num_results = min(k, len(chunks))
    scores, idxs = index.search(q, num_results)
    
    if len(idxs[0]) == 0:
        return [], 0.0
    
    # Filter by minimum score threshold and get docs
    docs = []
    valid_scores = []
    for i, idx in enumerate(idxs[0]):
        if scores[0][i] > 0.1:  # Minimum relevance threshold
            docs.append(chunks[idx])
            valid_scores.append(scores[0][i])
    
    # Return average of valid scores
    avg_score = float(np.mean(valid_scores)) if valid_scores else 0.0
    return docs, avg_score


In [ ]:
def self_heal(query):
    texts = []
    
    # Try Wikipedia API first (more reliable and clean)
    try:
        # Try direct page lookup first
        title_encoded = query.replace(" ", "_")
        api_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title_encoded}"
        resp = requests.get(api_url, headers=HEADERS, timeout=5)
        
        if resp.status_code == 200:
            data = resp.json()
            extract = data.get("extract", "")
            if extract and len(extract) > 100:
                texts.append(extract)
        else:
            # If direct lookup fails, try searching Wikipedia via DuckDuckGo
            # to find the correct Wikipedia page
            try:
                with DDGS() as ddgs:
                    wiki_results = list(ddgs.text(f"{query} site:wikipedia.org", max_results=2))
                    for result in wiki_results:
                        url = result.get("href", "")
                        if "wikipedia.org/wiki/" in url:
                            # Extract page title from URL
                            page_title = url.split("/wiki/")[-1].split("#")[0].split("?")[0]
                            # Get summary for this page
                            summary_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title}"
                            summary_resp = requests.get(summary_url, headers=HEADERS, timeout=5)
                            if summary_resp.status_code == 200:
                                summary_data = summary_resp.json()
                                extract = summary_data.get("extract", "")
                                if extract and len(extract) > 100:
                                    texts.append(extract)
                                    break  # Use first good result
            except:
                pass
    except Exception as e:
        pass
    
    # Web scraping as fallback (but with better cleaning)
    if not texts:
        try:
            with DDGS() as ddgs:
                for r in ddgs.text(query, max_results=3):
                    try:
                        resp = requests.get(r["href"], headers=HEADERS, timeout=8)
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, "html.parser")
                            
                            # Try to get main content areas
                            for tag in soup.find_all(['main', 'article', 'div'], class_=re.compile(r'content|main|article|post', re.I)):
                                text = tag.get_text()
                                cleaned = clean_text(text)
                                if cleaned and len(cleaned) > 200:
                                    texts.append(cleaned)
                                    break
                            else:
                                # Fallback to body text
                                text = soup.get_text()
                                cleaned = clean_text(text)
                                if cleaned and len(cleaned) > 200:
                                    texts.append(cleaned)
                    except:
                        continue
                    if len(texts) >= 2:  # Limit to avoid too much noise
                        break
        except:
            pass

    heal_chunks = []
    for t in texts:
        if t and len(t.strip()) > 50:
            chunks = chunk_text(t)
            heal_chunks.extend(chunks)

    return heal_chunks


In [13]:
def build_heal_index(heal_chunks):
    if not heal_chunks:
        return None, None

    emb = embedder.encode(heal_chunks).astype("float32")
    faiss.normalize_L2(emb)

    index = faiss.IndexFlatIP(emb.shape[1])
    index.add(emb)

    return index, heal_chunks


In [ ]:
def autorag_with_diff(query, threshold=0.5):
    # BEFORE: base knowledge only
    before_docs, score_before = retrieve_from(
        base_index, base_chunks, query, k=5
    )

    print("Trust score (before):", round(score_before, 3))

    after_docs = before_docs.copy() if before_docs else []
    score_after = score_before

    # Heal ONLY if needed
    if score_before < threshold:
        print("⚠️ Self-healing triggered...")
        heal_chunks = self_heal(query)
        
        if heal_chunks:
            heal_index, heal_chunks_list = build_heal_index(heal_chunks)

            if heal_index and heal_chunks_list:
                heal_docs, score_heal = retrieve_from(
                    heal_index, heal_chunks_list, query, k=5
                )
                
                # Hybrid approach: combine base and healed results
                if score_heal > score_before or score_before < 0.3:
                    # Use healed results if they're better, or if base is very poor
                    after_docs = heal_docs
                    score_after = score_heal
                    print(f" Using healed results (score: {round(score_heal, 3)})")
                else:
                    # Combine both if base is decent
                    combined_docs = (before_docs[:2] + heal_docs[:2])[:5]
                    after_docs = combined_docs
                    score_after = (score_before + score_heal) / 2
                    print(f" Combined base + healed results")
        else:
            print("⚠️ Self-healing failed - no additional content found")

    print("Trust score (after):", round(score_after, 3))

    before_text = clean_answer(" ".join(before_docs)) if before_docs else "No relevant information found."
    after_text = clean_answer(" ".join(after_docs)) if after_docs else "No relevant information found."

    return before_text, after_text


In [15]:
before, after = autorag_with_diff(
    "What are the latest AI regulations in Europe?"
)

print("\n--- BEFORE ---\n", before)
print("\n--- AFTER ---\n", after)


Trust score (before): 0.43
⚠️ Self-healing triggered
Trust score (after): 0.43

--- BEFORE ---
 acts as an agency of EASA on these issues , retaining its original regulatory powers in areas not yet transferred to EASA. Proposed developments seek to establish EASA as the single authority throughout the EU , taking over from individual member states the power to regulate all aviation other than that specifically excluded from the scope of EASA. in Europe with support from Aeon Zen and Anneke van Giersbergen

--- AFTER ---
 light in the darkest months tech sections artificial intelligence social media top stories tiktok signs deal to form new us unit with investors, including oracle, silver lake one tech tip: spend quality time with loved ones, not a screen, over the holidays militant groups are experimenting with ai, and the risks are expected to grow lifestyle sections food & recipes gardening homes travel fashion pets top stories '6-7,' dubai chocolate, labubu: you took 2025, now scat!

In [16]:
before, after = autorag_with_diff(
    "What is quantum computing?"
)

print("\n--- BEFORE ---\n", before)
print("\n--- AFTER ---\n", after)


Trust score (before): 0.26
⚠️ Self-healing triggered
Trust score (after): 0.81

--- BEFORE ---
 = = = Memory = = = = = Human interactions = = = = Mechanism = =

--- AFTER ---
 what is quantum computing? | ibm quantum what is quantum computing? authors josh schneider staff writer ibm think ian smalley staff editor ibm think what is quantum computing? quantum computing is an emergent field of computer science and engineering that harnesses the unique qualities of quantum mechanics to solve problems beyond the ability of even the most powerful classical computers. the field of quantum computing includes a range of disciplines, including quantum hardware and quantum algor what is quantum computing? | university of oxford department of physics skip to main content search research our research our research groups our research in action research funding support summer internships for undergraduates study undergraduates postgraduates engage for alumni for business for schools for the public me

In [17]:
before, after = autorag_with_diff(
    "How does OAuth authentication work?"
)

print("\n--- BEFORE ---\n", before)
print("\n--- AFTER ---\n", after)


Trust score (before): 0.23
⚠️ Self-healing triggered
Trust score (after): 0.23

--- BEFORE ---
 People who wanted information or advice consulted oracles , run by temples , that were supposed to convey gods ' answers to questions. Amulets and other images of protective deities were used to ward off the demons that might threaten human well @-@ being or to impart the god 's positive characteristics to the wearer. Private rituals invoked the gods ' power to accomplish personal goals , from healing sickness to cursing enemies

--- AFTER ---
 People who wanted information or advice consulted oracles , run by temples , that were supposed to convey gods ' answers to questions. Amulets and other images of protective deities were used to ward off the demons that might threaten human well @-@ being or to impart the god 's positive characteristics to the wearer. Private rituals invoked the gods ' power to accomplish personal goals , from healing sickness to cursing enemies
